# "La Famille BERT et les Transformers"
> "Qu'est-ce que BERT ? Quel lien avec les transformers ? Pourquoi les utilise-t-on ? Comment ?"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [fr, nlp]
- hide: false
- search_exclude: false


In [7]:
%pip install --upgrade -q pandas torch transformers "spacy>=3.2.0" spacy-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/Users/yco/.pyenv/versions/proto-38dev/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Ressources
### Références
* Attention is all you need (2016, Transformer paper): https://arxiv.org/abs/1706.03762
* BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding (2018): https://arxiv.org/abs/1810.04805


* Illustrated transformer: http://jalammar.github.io/illustrated-transformer/
* BERT model doc on hugging face: https://huggingface.co/transformers/model_doc/bert.html#bertmodel
* https://spacy.io/usage/embeddings-transformers

### Exemples
* Exemple de configuration sur le hub de huggingface: https://huggingface.co/dslim/bert-base-NER/blob/main/config.json
* Transformers pour le NER FR https://huggingface.co/models?language=fr&pipeline_tag=token-classification&sort=downloads&search=ner
* Facebook Language AI Research NER: https://huggingface.co/flair/ner-french (Org, Per, Loc, Names)
* https://towardsdatascience.com/easy-fine-tuning-of-transformers-for-named-entity-recognition-d72f2b5340e3
* https://skimai.com/how-to-fine-tune-bert-for-named-entity-recognition-ner/
* Use huggingface transformers within spacy: https://reposhub.com/python/deep-learning/explosion-spacy-transformers.html

### Cours et ressources formatrices
* https://www.coursera.org/learn/attention-models-in-nlp/home/welcome
* https://huggingface.co/course/chapter1
* https://course.spacy.io/en/chapter4
* https://www.youtube.com/playlist?list=PL75e0qA87dlG-za8eLI6t0_Pbxafk-cxb

# Why Transformers / BERT ?
* Les Transformers s'appuient sur le mécanisme d'attention, qui a prouvé son efficacité
* Les Transformers sont une architecture de deep learning générique, applicable à toutes les tâches de NLP
* Les tâches de NLP (classification, NER, traduction, QA, ...) se ressemblent beaucoup, et une grande partie de l'effort est commun: comprendre la structure d'un langage, les relations entre les mots etc.
    * Il y a un intérêt important à avoir un moyen de mutualiser cette approche pour transférer la connaissance
    * Le word / sentence embeddings sont un premier pas dans ce sens, mais pas sur les relations entre les mots ni sur le sens des phrases

# What are they ?
* Un transformer est une architecture de deep learning qui utilise le mécanisme d'attention: l'importance de chaque mot de la phrase par rapport à chaque autre mot est prise en compte

![attention](bert/attention.png "Mécanisme d'attention")
![scaled dot prod](bert/scaleddotprod.png)

* L'architecture Transformer est composée d'un encoder et d'un decoder, tous deux basés sur le mécanisme d'attention. Dans le décodeur l'attention est dite causale (ou masquée), c'est à dire qu'on ne regarde que les mots passés.

![transformer](bert/transformer_archi.png)
![multi head attn](bert/multihead.png)

* BERT est initialement une approche de pre-training pour capturer les relations dans un langage, l'entraînement se fait sur une tâche auto-supervisée (self-supervised) ce qui permet de l'entraîner sur de grande quantité de données. Le fine tuning se fait ensuite en utilisant le transfer learning

![bert](bert/bert_training.png)

# How to use Transformers
BERT et les autres transformers sont conçus pour être réutilisable en grande partie pour différentes tâche, il ne reste donc qu'à réadapter les entrées et sorties à la tâche visée

## 🤗 Transformers
Hugging Face a construit toute une API pour faciliter l'utilisation des transformers. La librairie propose des pipelines pré-définies et une bibliothèque de modèles et de datasets prêtes à l'emplois.

La configuration d'un modèle se fait très facilement (sans code) à l'aide d'un fichier de configuration: https://huggingface.co/dslim/bert-base-NER/blob/main/config.json

```json
{
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996,
  # Labels
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  }
}
```

Ici l'architecture choisie est `BertForTokenClassification` ce qui correspond à une tâche de NER (affecter à chaque token un label). Et on retrouve aussi les différents hyper-paramètres des transformers (nombre de têtes d'attention, paramètres d'attention, ...). Pour pouvoir réutiliser les poids d'entraînement d'un modèle pré-entraîné, il ne faut faire attention à ne changer que ce qui ne casse pas la compatibilité avec le modèle pré-entraîné.

Note: Les labels sont définis avec le mode d'anotation "IOB" (I: Inside entity, O: Outside entity, B: Beginning of entity) ex: Je suis Yoann Couble serait annoté "O O B-PER I-PER".

### Solution 1: use a pre-defined / pre-trained model

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_name = "dslim/bert-base-NER"

#### Get model and associated tokenizer from model hub

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

#### Define pipeline

In [4]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

#### Use on example

In [5]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'word': 'Wolfgang', 'score': 0.9990139603614807, 'entity': 'B-PER', 'index': 4, 'start': 11, 'end': 19}, {'word': 'Berlin', 'score': 0.9996449947357178, 'entity': 'B-LOC', 'index': 9, 'start': 34, 'end': 40}]


### Solution 2: use the raw BERT model and customize its heads

Start from a [raw BERT model without any head](https://huggingface.co/transformers/model_doc/bert.html#bertmodel), but where the weights can be re-used.
Create a child class and define the `__init__` and `forward` methods, see [example from BertForTokenClassification](https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForTokenClassification).

#### Example from BertForTokenClassification

```python
class BertForTokenClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        ...
        # See doc linked above
```

## Within Spacy
Spacy est complètement intégré dans huggingface, il est donc possible de les réutiliser à l'intérieur des pipelines spacy.

Spacy fourni également des modèles pré-entraînés et bien intégrés à Spacy. Ce sont les modèles finissant en `_trf` comme [celui-ci basé sur camembert-base](https://spacy.io/models/fr#fr_dep_news_trf)

Download the model within your virtual environment / kernel (in my case, it's a pyenv environment)

```bash
pyenv activate 3.8.6
python -m spacy download en_core_web_trf
```

In [11]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_trf")
doc = nlp("My name is Wolfgang and I live in Berlin")

displacy.render(doc, "ent")